### Creating the tables

In [1]:
import pandas as pd

In [13]:
customers = pd.DataFrame([(1,"Yeonjoo", "somewhere in USA", "123456")],columns =["customerID", "name", "address", "phone"])

In [14]:
customers

,customerID,name,address,phone
0,1,Yeonjoo,somewhere in USA,123456


In [15]:
customers.index = customers.customerID

In [20]:
customers.drop('customerID', axis=1, inplace=True)

In [21]:
customers

,name,address,phone
customerID,,,
1,Yeonjoo,somewhere in USA,123456


In [30]:
orders = pd.DataFrame([(1,"04/07/2020", "toilet paper", "1000", 1)],
                         columns =["order_number", "date", "product_name", "quantity", "customerID"])

In [31]:
orders.index = orders.order_number

In [32]:
orders.drop('order_number', axis=1, inplace=True)

In [33]:
orders

,date,product_name,quantity,customerID
order_number,,,,
1,04/07/2020,toilet paper,1000,1


In [38]:
import pandas as pd 
from pandasql import sqldf

In [39]:
pysqldf = lambda q: sqldf(q, globals())

In [44]:
df = pd.read_csv('titanic.csv')
df.drop(df.columns[0], axis =1, inplace = True)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Examples for pandasql

### Fares between 50 and 100

In [45]:
fares_50_to_100_df = df[(50<df.Fare) & (df.Fare<100)]
fares_50_to_100_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
34,35,0,1,"Meyer, Mr. Edgar Joseph",male,28.0,1,0,PC 17604,82.1708,NaN,C
35,36,0,1,"Holverson, Mr. Alexander Oskar",male,42.0,1,0,113789,52.0000,NaN,S


In [58]:
q= """
        select  *
        from df
        where (Fare >50) & (Fare<100) 

        """

fares_50_to_100_df_sql = pysqldf(q) 
fares_50_to_100_df_sql.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
1,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
2,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
3,35,0,1,"Meyer, Mr. Edgar Joseph",male,28.0,1,0,PC 17604,82.1708,None,C
4,36,0,1,"Holverson, Mr. Alexander Oskar",male,42.0,1,0,113789,52.0000,None,S


In [50]:
fares_50_to_100_df.equals(fares_50_to_100_df_sql)

False

In [54]:
fares_50_to_100_df_sql.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
1,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
2,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
3,35,0,1,"Meyer, Mr. Edgar Joseph",male,28.0,1,0,PC 17604,82.1708,None,C
4,36,0,1,"Holverson, Mr. Alexander Oskar",male,42.0,1,0,113789,52.0000,None,S


In [57]:
fares_50_to_100_df.set_index('PassengerId').equals(fares_50_to_100_df_sql.set_index('PassengerId'))

True

### Male passengers who survived

In [71]:
#Using pandas
surviving_males_df = df[(df.Sex == 'male')&(df.Survived ==1)][['Name','Fare']][:30]

#using SQL query 
q2 = """
        select Name, Fare
        from df
        where
        (Sex = 'male') & (Survived = 1)
        limit 30
            
        """

surviving_males_sql = pysqldf(q2)

surviving_males_df.set_index('Name').equals(surviving_males_sql.set_index('Name'))


True

## Examples of `.query()`

In [73]:
female_children_df = df[(df.Sex=='female')&(df.Age <15)]

In [74]:
female_children_query = df.query("Sex == 'female' &  Age<15")

In [75]:
female_children_query.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
14,15,0,3,"Vestrom, Miss. Hulda Amanda Adolfina",female,14.0,0,0,350406,7.8542,NaN,S
24,25,0,3,"Palsson, Miss. Torborg Danira",female,8.0,3,1,349909,21.0750,NaN,S
39,40,1,3,"Nicola-Yarred, Miss. Jamila",female,14.0,1,0,2651,11.2417,NaN,C


In [77]:
female_children_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
14,15,0,3,"Vestrom, Miss. Hulda Amanda Adolfina",female,14.0,0,0,350406,7.8542,NaN,S
24,25,0,3,"Palsson, Miss. Torborg Danira",female,8.0,3,1,349909,21.0750,NaN,S
39,40,1,3,"Nicola-Yarred, Miss. Jamila",female,14.0,1,0,2651,11.2417,NaN,C


In [78]:
female_children_df.equals(female_children_query) 

True